In [25]:
# lib 

from urllib.request import urlopen
import bs4
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import RobustScaler

# Crawling - Before COVID-19

In [ ]:
#url

page_n = 1
USD_index = f"https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW&page={page_n}"
USD_index

# source 설정
source = urlopen(USD_index).read()
source = bs4.BeautifulSoup(source, 'lxml')
source

<html lang="ko">
<head>
<title>네이버 금융</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220211144316/css/finance.css" rel="stylesheet" type="text/css"/>
<script language="javascript">document.domain="naver.com";</script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20220211144316/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20220211144316/js/lcslog.js" type="text/javascript"></script>
</head>
<body>
<div class="section_exchange section_today">
<h3 class="h_today"><span>일별 환율</span></h3>
<table border="1" class="tbl_exchange today" summary="일별 환율 리스트">
<caption>일별 환율</caption>
<colgroup>
<col width="78"/>
<col width="78"/>
<col width="73"/>
<col width="76"/>
<col width="76"/>
<col width="76

In [ ]:
# date func

def date_format(date):
    yyyy, mm, dd = [int(x) for x in date.split(".")]
    return dt.date(yyyy, mm, dd)

In [ ]:
# WTI

dates = source.find_all('td', class_ = 'date')
prices = source.find_all('td', class_ = 'num')

def crawl_WTI_naver_index(end_page):
    WTI_date_list = []
    WTI_price_list = []
    
    for page_n in range(72, end_page+1):
        
        WTI_naver_index = f"https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=OIL_CL&fdtc=2&page={page_n}"
        
        source = urlopen(WTI_naver_index).read()
        source = bs4.BeautifulSoup(source, 'lxml')
        
        dates = source.find_all('td', class_ = 'date')
        prices = source.find_all('td', class_ = 'num')
        
        for i in range(len(dates)):
            this_date = dates[i].text
            this_date = date_format(this_date)
            
            this_price = prices[i*3].text
            this_price = float(this_price)
            
            WTI_date_list.append(this_date)
            WTI_price_list.append(this_price)
            
    df = pd.DataFrame({"날짜" : WTI_date_list, "가격(지수)" : WTI_price_list})
    return df

    
WTI_df = crawl_WTI_naver_index(443)   # 73~443
# WTI_df = WTI_df.drop(index = [0, 1, 2596], axis = 0)
WTI_df.set_index('날짜', inplace = True)
WTI_df

# WTI_df.to_csv("BF_WTI_data.csv", mode = "w", index = False, encoding = 'utf-8-sig')

,가격(지수)
날짜,
2020-03-09,31.13
2020-03-06,41.28
2020-03-05,45.90
2020-03-04,46.78
2020-03-03,47.18
...,...
2010-01-04,81.51
2009-12-31,79.36
2009-12-30,79.28


In [ ]:
# KOSPI

dates = source.find_all('td', class_ = 'date')
KOSPI_IDXs = source.find_all('td', class_ = 'number_1')

def crawl_KOSPI_naver_index(end_page):
    KOSPI_date_list = []
    KOSPI_IDXs_list = []
    
    for page_n in range(83, end_page+1):
        
        KOSPI_naver_index = f"https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page={page_n}"
        
        source = urlopen(KOSPI_naver_index).read()
        source = bs4.BeautifulSoup(source, 'lxml')
        
        dates = source.find_all('td', class_ = 'date')
        KOSPI_IDXs = source.find_all('td', class_ = 'number_1')
        
        for i in range(len(dates)):
            this_date = dates[i].text
            this_date = date_format(this_date)
            
            this_KOSPI_IDXs = KOSPI_IDXs[i*4].text.replace(",", "")
            this_KOSPI_IDXs = float(this_KOSPI_IDXs)
            
            KOSPI_date_list.append(this_date)
            KOSPI_IDXs_list.append(this_KOSPI_IDXs)
            
    df = pd.DataFrame({"날짜" : KOSPI_date_list, "가격(지수)" : KOSPI_IDXs_list})
    return df

    
KOSPI_df = crawl_KOSPI_naver_index(500)
# KOSPI_df = KOSPI_df.drop(index = [0, 2507], axis = 0)
KOSPI_df.set_index('날짜', inplace = True)
KOSPI_df
# KOSPI_df.to_csv("BF_KOSPI_data.csv", mode = "w", index = False, encoding = 'utf-8-sig')

,가격(지수)
날짜,
2020-03-03,2014.15
2020-03-02,2002.51
2020-02-28,1987.01
2020-02-27,2054.89
2020-02-26,2076.77
...,...
2010-01-08,1695.26
2010-01-07,1683.45
2010-01-06,1705.32


In [ ]:
# GOLD

dates = source.find_all('td', class_ = 'date')
prices = source.find_all('td', class_ = 'num')

def crawl_GOLD_naver_index(end_page):
    GOLD_date_list = []
    GOLD_price_list = []
    
    for page_n in range(71, end_page+1):
        
        GOLD_naver_index = f"https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_GC&fdtc=2&page={page_n}"
        
        source = urlopen(GOLD_naver_index).read()
        source = bs4.BeautifulSoup(source, 'lxml')
        
        dates = source.find_all('td', class_ = 'date')
        prices = source.find_all('td', class_ = 'num')
        
        for i in range(len(dates)):
            this_date = dates[i].text
            this_date = date_format(this_date)
            
            this_price = prices[i*3].text.replace(",", "")
            this_price = float(this_price)
            
            GOLD_date_list.append(this_date)
            GOLD_price_list.append(this_price)
            
    df = pd.DataFrame({"날짜" : GOLD_date_list, "가격(지수)" : GOLD_price_list})
    return df

    
GOLD_df = crawl_GOLD_naver_index(442)
# GOLD_df = GOLD_df.drop(index = [0, 1, 2588, 2589], axis = 0)
GOLD_df.set_index('날짜', inplace = True)
GOLD_df

,가격(지수)
날짜,
2020-03-18,1477.3
2020-03-17,1524.9
2020-03-16,1485.9
2020-03-13,1515.7
2020-03-12,1589.3
...,...
2009-12-31,1095.2
2009-12-30,1091.5
2009-12-29,1097.0


In [ ]:
# government bond

dates = source.find_all('td', class_ = 'date')
prices = source.find_all('td', class_ = 'num')

def crawl_TB_naver_index(end_page):
    TB_date_list = []
    TB_price_list = []
    
    for page_n in range(70, end_page+1):
        
        TB_naver_index = f"https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_GOVT03Y&page={page_n}"
    
        
        source = urlopen(TB_naver_index).read()
        source = bs4.BeautifulSoup(source, 'lxml')
        
        dates = source.find_all('td', class_ = 'date')
        prices = source.find_all('td', class_ = 'num')
        
        for i in range(len(dates)):
            this_date = dates[i].text
            this_date = date_format(this_date)
            
            this_price = prices[i*3].text
            this_price = float(this_price)
            
            TB_date_list.append(this_date)
            TB_price_list.append(this_price)
            
    df = pd.DataFrame({"날짜" : TB_date_list, "가격(지수)" : TB_price_list})
    return df

    
TB_df = crawl_TB_naver_index(430)    #  51~97P
# TB_df = TB_df.drop(index = [2512], axis = 0)
TB_df.set_index('날짜', inplace = True)
TB_df

# TB_df.to_csv("BF_TB_data.csv", mode = 'w', index = False, encoding = 'utf-8-sig')

,가격(지수)
날짜,
2020-03-12,1.06
2020-03-11,1.09
2020-03-10,1.08
2020-03-09,1.04
2020-03-06,1.08
...,...
2009-12-30,4.41
2009-12-29,4.39
2009-12-28,4.36


In [ ]:
# USD

dates = source.find_all('td', class_ = 'date')
prices = source.find_all('td', class_ = 'num')

def crawl_USD_naver_index(end_page):
    USD_date_list = []
    USD_price_list = []
    
    for page_n in range(49, end_page+1):
        
        USD_naver_index = f"https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW&page={page_n}"
    
        
        source = urlopen(USD_naver_index).read()
        source = bs4.BeautifulSoup(source, 'lxml')
        
        dates = source.find_all('td', class_ = 'date')
        prices = source.find_all('td', class_ = 'num')
        
        for i in range(len(dates)):
            this_date = dates[i].text
            this_date = date_format(this_date)
            
            this_price = prices[i*2].text.replace(",", "")
            this_price = float(this_price)
            
            USD_date_list.append(this_date)
            USD_price_list.append(this_price)
            
    df = pd.DataFrame({"날짜" : USD_date_list, "가격(지수)" : USD_price_list})
    return df

    
USD_df = crawl_USD_naver_index(302)   
# USD_df = USD_df.drop(index = [0, 1, 2, 3, 4, 5, 2522, 2523, 2524, 2525, 2526, 2527, 2528, 2529], axis = 0)
USD_df.set_index('날짜', inplace = True)
USD_df
# USD_df.to_csv("BF_USD_data.csv", mode = "w", index = False, encoding = 'utf-8-sig')

,가격(지수)
날짜,
2020-03-23,1273.0
2020-03-20,1245.0
2020-03-19,1280.0
2020-03-18,1257.0
2020-03-17,1242.0
...,...
2009-12-28,1170.5
2009-12-24,1176.0
2009-12-23,1184.5


In [ ]:
bf_WTI_df = WTI_df.reset_index().drop(index = [0, 1, 2, 3, 4 ,5, 2600, 2601, 2602, 2603], axis = 0)
bf_WTI_df

,날짜,가격(지수)
6,2020-02-28,44.76
7,2020-02-27,47.09
8,2020-02-26,48.73
9,2020-02-25,49.90
10,2020-02-24,51.43
...,...,...
2595,2010-01-08,82.75
2596,2010-01-07,82.66
2597,2010-01-06,83.18
2598,2010-01-05,81.77


In [ ]:
bf_KOSPI_df = KOSPI_df.reset_index().drop(index = [0, 1], axis = 0)
bf_KOSPI_df

,날짜,가격(지수)
2,2020-02-28,1987.01
3,2020-02-27,2054.89
4,2020-02-26,2076.77
5,2020-02-25,2103.61
6,2020-02-24,2079.04
...,...,...
2503,2010-01-08,1695.26
2504,2010-01-07,1683.45
2505,2010-01-06,1705.32
2506,2010-01-05,1690.62


In [ ]:
bf_GOLD_df = GOLD_df.reset_index().drop(index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                    2599, 2600, 2601, 2602, 2603], axis = 0)
bf_GOLD_df

,날짜,가격(지수)
13,2020-02-28,1564.1
14,2020-02-27,1640.0
15,2020-02-26,1640.4
16,2020-02-25,1646.9
17,2020-02-24,1672.4
...,...,...
2594,2010-01-08,1138.2
2595,2010-01-07,1133.1
2596,2010-01-06,1135.9
2597,2010-01-05,1118.1


In [ ]:
bf_TB_df = TB_df.reset_index().drop(index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 2526, 2525, 2524, 2523, 2522, 2521], axis = 0)
bf_TB_df

,날짜,가격(지수)
9,2020-02-28,1.10
10,2020-02-27,1.19
11,2020-02-26,1.14
12,2020-02-25,1.17
13,2020-02-24,1.14
...,...,...
2516,2010-01-08,4.36
2517,2010-01-07,4.32
2518,2010-01-06,4.43
2519,2010-01-05,4.36


In [ ]:
bf_USD_df = USD_df.reset_index().drop(index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 2539, 2538, 2537, 2536, 2535, 2534, 2533, 2532], axis = 0)
bf_USD_df

,날짜,가격(지수)
16,2020-02-28,1210.5
17,2020-02-27,1212.0
18,2020-02-26,1215.0
19,2020-02-25,1215.0
20,2020-02-24,1219.0
...,...,...
2527,2010-01-08,1132.0
2528,2010-01-07,1135.5
2529,2010-01-06,1138.5
2530,2010-01-05,1140.5


In [ ]:
df_list = ['bf_WTI_df', 'bf_KOSPI_df', 'bf_GOLD_df', 'bf_TB_df', 'bf_USD_df']

a = pd.merge(bf_WTI_df, bf_KOSPI_df, on = '날짜', how = 'inner')

b = pd.merge(a, bf_GOLD_df, on = '날짜', how = 'inner')
b.rename(columns = 
         {'가격(지수)_x' : 'WTI', '가격(지수)_y' : 'KOSPI', '가격(지수)' : 'GOLD'}, inplace = True)

c = pd.merge(b, bf_TB_df, on = '날짜', how = 'inner')

bf_df = pd.merge(c, bf_USD_df, on = '날짜', how = 'inner')

bf_df.rename(columns = 
         {'가격(지수)_x': 'TB', '가격(지수)_y' : 'USD'}, inplace = True)

bf_df.set_index('날짜', inplace = True)
bf_df

,WTI,KOSPI,GOLD,TB,USD
날짜,,,,,
2020-02-28,44.76,1987.01,1564.1,1.10,1210.5
2020-02-27,47.09,2054.89,1640.0,1.19,1212.0
2020-02-26,48.73,2076.77,1640.4,1.14,1215.0
2020-02-25,49.90,2103.61,1646.9,1.17,1215.0
2020-02-24,51.43,2079.04,1672.4,1.14,1219.0
...,...,...,...,...,...
2010-01-08,82.75,1695.26,1138.2,4.36,1132.0
2010-01-07,82.66,1683.45,1133.1,4.32,1135.5
2010-01-06,83.18,1705.32,1135.9,4.43,1138.5


# Crawling - After COVID-19

In [ ]:
# WTI

dates = source.find_all('td', class_ = 'date')
prices = source.find_all('td', class_ = 'num')

def crawl_WTI_naver_index(end_page):
    WTI_date_list = []
    WTI_price_list = []
    
    for page_n in range(0, end_page+1):
        
        WTI_naver_index = f"https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=OIL_CL&fdtc=2&page={page_n}"
        
        source = urlopen(WTI_naver_index).read()
        source = bs4.BeautifulSoup(source, 'lxml')
        
        dates = source.find_all('td', class_ = 'date')
        prices = source.find_all('td', class_ = 'num')
        
        for i in range(len(dates)):
            this_date = dates[i].text
            this_date = date_format(this_date)
            
            this_price = prices[i*3].text
            this_price = float(this_price)
            
            WTI_date_list.append(this_date)
            WTI_price_list.append(this_price)
            
    df = pd.DataFrame({"날짜" : WTI_date_list, "가격(지수)" : WTI_price_list})
    return df

    
WTI_df = crawl_WTI_naver_index(72)   # 0~72
# WTI_df = WTI_df.drop(index = [0, 1, 2596], axis = 0)
# WTI_df.set_index('날짜', inplace = True)
WTI_df

# WTI_df.to_csv("BF_WTI_data.csv", mode = "w", index = False, encoding = 'utf-8-sig')

,날짜,가격(지수)
0,2022-02-25,91.59
1,2022-02-24,92.81
2,2022-02-23,92.10
3,2022-02-22,91.91
4,2022-02-18,90.21
...,...,...
499,2020-03-05,45.90
500,2020-03-04,46.78
501,2020-03-03,47.18
502,2020-03-02,46.75


In [ ]:
# KOSPI

dates = source.find_all('td', class_ = 'date')
KOSPI_IDXs = source.find_all('td', class_ = 'number_1')

def crawl_KOSPI_naver_index(end_page):
    KOSPI_date_list = []
    KOSPI_IDXs_list = []
    
    for page_n in range(0, end_page+1):
        
        KOSPI_naver_index = f"https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page={page_n}"
        
        source = urlopen(KOSPI_naver_index).read()
        source = bs4.BeautifulSoup(source, 'lxml')
        
        dates = source.find_all('td', class_ = 'date')
        KOSPI_IDXs = source.find_all('td', class_ = 'number_1')
        
        for i in range(len(dates)):
            this_date = dates[i].text
            this_date = date_format(this_date)
            
            this_KOSPI_IDXs = KOSPI_IDXs[i*4].text.replace(",", "")
            this_KOSPI_IDXs = float(this_KOSPI_IDXs)
            
            KOSPI_date_list.append(this_date)
            KOSPI_IDXs_list.append(this_KOSPI_IDXs)
            
    df = pd.DataFrame({"날짜" : KOSPI_date_list, "가격(지수)" : KOSPI_IDXs_list})
    return df

    
KOSPI_df = crawl_KOSPI_naver_index(84)
# KOSPI_df = KOSPI_df.drop(index = [0, 2507], axis = 0)
# KOSPI_df.set_index('날짜', inplace = True)
KOSPI_df
# KOSPI_df.to_csv("BF_KOSPI_data.csv", mode = "w", index = False, encoding = 'utf-8-sig')

,날짜,가격(지수)
0,2022-02-28,2699.18
1,2022-02-25,2676.76
2,2022-02-24,2648.80
3,2022-02-23,2719.53
4,2022-02-22,2706.79
...,...,...
499,2020-02-21,2162.84
500,2020-02-20,2195.50
501,2020-02-19,2210.34
502,2020-02-18,2208.88


In [ ]:
# GOLD

dates = source.find_all('td', class_ = 'date')
prices = source.find_all('td', class_ = 'num')

def crawl_GOLD_naver_index(end_page):
    GOLD_date_list = []
    GOLD_price_list = []
    
    for page_n in range(0, end_page+1):
        
        GOLD_naver_index = f"https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_GC&fdtc=2&page={page_n}"
        
        source = urlopen(GOLD_naver_index).read()
        source = bs4.BeautifulSoup(source, 'lxml')
        
        dates = source.find_all('td', class_ = 'date')
        prices = source.find_all('td', class_ = 'num')
        
        for i in range(len(dates)):
            this_date = dates[i].text
            this_date = date_format(this_date)
            
            this_price = prices[i*3].text.replace(",", "")
            this_price = float(this_price)
            
            GOLD_date_list.append(this_date)
            GOLD_price_list.append(this_price)
            
    df = pd.DataFrame({"날짜" : GOLD_date_list, "가격(지수)" : GOLD_price_list})
    return df

    
GOLD_df = crawl_GOLD_naver_index(73)
# GOLD_df = GOLD_df.drop(index = [0, 1, 2588, 2589], axis = 0)
# GOLD_df.set_index('날짜', inplace = True)
GOLD_df

,날짜,가격(지수)
0,2022-02-25,1886.5
1,2022-02-24,1925.4
2,2022-02-23,1909.4
3,2022-02-22,1906.4
4,2022-02-18,1898.8
...,...,...
506,2020-02-25,1646.9
507,2020-02-24,1672.4
508,2020-02-21,1644.6
509,2020-02-20,1616.6


In [ ]:
# government bond

dates = source.find_all('td', class_ = 'date')
prices = source.find_all('td', class_ = 'num')

def crawl_TB_naver_index(end_page):
    TB_date_list = []
    TB_price_list = []
    
    for page_n in range(0, end_page+1):
        
        TB_naver_index = f"https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_GOVT03Y&page={page_n}"
    
        
        source = urlopen(TB_naver_index).read()
        source = bs4.BeautifulSoup(source, 'lxml')
        
        dates = source.find_all('td', class_ = 'date')
        prices = source.find_all('td', class_ = 'num')
        
        for i in range(len(dates)):
            this_date = dates[i].text
            this_date = date_format(this_date)
            
            this_price = prices[i*3].text
            this_price = float(this_price)
            
            TB_date_list.append(this_date)
            TB_price_list.append(this_price)
            
    df = pd.DataFrame({"날짜" : TB_date_list, "가격(지수)" : TB_price_list})
    return df

    
TB_df = crawl_TB_naver_index(71)    #  51~97P
# TB_df = TB_df.drop(index = [2512], axis = 0)
# TB_df.set_index('날짜', inplace = True)
TB_df

# TB_df.to_csv("BF_TB_data.csv", mode = 'w', index = False, encoding = 'utf-8-sig')

,날짜,가격(지수)
0,2022-02-28,2.24
1,2022-02-25,2.24
2,2022-02-24,2.22
3,2022-02-23,2.31
4,2022-02-22,2.32
...,...,...
492,2020-02-28,1.10
493,2020-02-27,1.19
494,2020-02-26,1.14
495,2020-02-25,1.17


In [ ]:
# USD

dates = source.find_all('td', class_ = 'date')
prices = source.find_all('td', class_ = 'num')

def crawl_USD_naver_index(end_page):
    USD_date_list = []
    USD_price_list = []
    
    for page_n in range(0, end_page+1):
        
        USD_naver_index = f"https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW&page={page_n}"
    
        
        source = urlopen(USD_naver_index).read()
        source = bs4.BeautifulSoup(source, 'lxml')
        
        dates = source.find_all('td', class_ = 'date')
        prices = source.find_all('td', class_ = 'num')
        
        for i in range(len(dates)):
            this_date = dates[i].text
            this_date = date_format(this_date)
            
            this_price = prices[i*2].text.replace(",", "")
            this_price = float(this_price)
            
            USD_date_list.append(this_date)
            USD_price_list.append(this_price)
            
    df = pd.DataFrame({"날짜" : USD_date_list, "가격(지수)" : USD_price_list})
    return df

    
USD_df = crawl_USD_naver_index(51)   
# USD_df = USD_df.drop(index = [0, 1, 2, 3, 4, 5, 2522, 2523, 2524, 2525, 2526, 2527, 2528, 2529], axis = 0)
# USD_df.set_index('날짜', inplace = True)
USD_df
# USD_df.to_csv("BF_USD_data.csv", mode = "w", index = False, encoding = 'utf-8-sig')

,날짜,가격(지수)
0,2022-02-28,1203.5
1,2022-02-25,1204.5
2,2022-02-24,1204.5
3,2022-02-23,1192.0
4,2022-02-22,1193.0
...,...,...
505,2020-02-17,1184.0
506,2020-02-14,1183.0
507,2020-02-13,1182.5
508,2020-02-12,1180.0


In [ ]:
af_WTI_df = WTI_df.drop(index = [503], axis = 0)

In [ ]:
af_KOSPI_df = KOSPI_df.drop(index = [0, 503, 502, 501, 500, 499, 498, 497, 496, 495, 494])

In [ ]:
af_GOLD_df = GOLD_df.drop(index = [510, 509, 508, 507, 506, 505, 504, 503])

In [ ]:
af_TB_df = TB_df.drop(index = [0, 496, 495, 494, 493, 492])

In [ ]:
af_USD_df = USD_df.drop(index = [0, 509, 508, 507, 506, 505, 504, 503, 502, 501, 500, 499, 498, 497, 496])

In [ ]:
af_df = pd.merge(af_WTI_df, af_KOSPI_df, on = '날짜', how = 'inner')

af_df = pd.merge(af_df, af_GOLD_df, on = '날짜', how = 'inner')
af_df.rename(columns = 
         {'가격(지수)_x' : 'WTI', '가격(지수)_y' : 'KOSPI', '가격(지수)' : 'GOLD'}, inplace = True)

af_df = pd.merge(af_df, af_TB_df, on = '날짜', how = 'inner')

af_df = pd.merge(af_df, af_USD_df, on = '날짜', how = 'inner')

af_df.rename(columns = 
         {'가격(지수)_x': 'TB', '가격(지수)_y' : 'USD'}, inplace = True)
af_df.set_index('날짜', inplace = True)
af_df

,WTI,KOSPI,GOLD,TB,USD
날짜,,,,,
2022-02-25,91.59,2676.76,1886.5,2.24,1204.5
2022-02-24,92.81,2648.80,1925.4,2.22,1204.5
2022-02-23,92.10,2719.53,1909.4,2.31,1192.0
2022-02-22,91.91,2706.79,1906.4,2.32,1193.0
2022-02-18,90.21,2744.52,1898.8,2.32,1195.5
...,...,...,...,...,...
2020-03-06,41.28,2040.22,1670.8,1.08,1191.0
2020-03-05,45.90,2085.26,1666.4,1.05,1184.5
2020-03-04,46.78,2059.33,1641.1,1.03,1183.0


In [ ]:
bf_df.to_csv("BF_data.csv", mode = "w", index = True, encoding = 'utf-8-sig')

In [ ]:
af_df.to_csv("AF_data.csv", mode = "w", index = True, encoding = 'utf-8-sig')

# Data Processing

In [5]:
af_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/시장지표/AF_data.csv')
af_data.drop('날짜', axis = 1, inplace = True)
bf_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/시장지표/BF_data.csv')
bf_data.drop('날짜', axis = 1, inplace = True)

In [6]:
# Outlier Index

def get_outlier(df=None, column=None, weight=2.0):
  quantile_25 = np.percentile(df[column].values, 25)
  quantile_75 = np.percentile(df[column].values, 75)

  IQR = quantile_75 - quantile_25
  IQR_weight = IQR*weight
  
  lowest = quantile_25 - IQR_weight
  highest = quantile_75 + IQR_weight
  
  outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
  return outlier_idx

In [7]:
columns = ['WTI', 'KOSPI', 'GOLD', 'TB', 'USD']

for col in columns:
    outlier_idx = get_outlier(bf_data, col, weight=1.5)
    bf_data.drop(outlier_idx, axis=0, inplace=True)

for col in columns:
    outlier_idx = get_outlier(af_data, col, weight=1.5)
    af_data.drop(outlier_idx, axis=0, inplace=True)

bf_data.reset_index(inplace = True, drop = True)
af_data.reset_index(inplace = True, drop = True)

In [8]:
print(bf_data.shape)
print(af_data.shape)

(2182, 5)
(449, 5)


In [9]:
data = pd.concat([bf_data, af_data], axis = 0)

scaler = RobustScaler()
scaler.fit(data)
data_scaled = pd.DataFrame(scaler.transform(data))

columns = ['WTI', 'KOSPI', 'GOLD', 'TB', 'USD']

data_scaled.columns = columns

bf_scaled = data_scaled.iloc[:2182]
af_scaled = data_scaled.iloc[2182:]
af_scaled.reset_index(drop = True, inplace = True)
bf_scaled.reset_index(drop = True, inplace = True)

print(bf_scaled.shape)
print(af_scaled.shape)

(2182, 5)
(449, 5)


# Visualization

In [10]:
def corr_heatmap(dataset, title):
    cmap = plt.cm.PuBu
    corr = dataset.corr()
    plt.figure(figsize=(20, 16)) 
    plt.title(title, y = 1.05, size = 25) 
    sns.heatmap(corr, linewidths = 0.1, 
                vmax = 1.0, 
                square = True, 
                cmap = cmap, 
                linecolor = "white", 
                annot = True, 
                annot_kws = {"size" : 16})

In [ ]:
corr_heatmap(data_scaled, 'Person Correlation - Total Dataset')

In [ ]:
corr_heatmap(bf_scaled, 'Person Correlation - Before COVID-19')

In [ ]:
corr_heatmap(af_scaled, 'Person Correlation - After COVID-19')

In [14]:
AF = af_scaled.corr()
BF = bf_scaled.corr()

diff = AF - BF
diff

,WTI,KOSPI,GOLD,TB,USD
WTI,0.000000,1.105423,-0.489863,0.019077,0.353700
KOSPI,1.105423,0.000000,0.223156,0.992125,-0.503157
GOLD,-0.489863,0.223156,0.000000,-0.531429,-0.215231
TB,0.019077,0.992125,-0.531429,0.000000,0.511515
USD,0.353700,-0.503157,-0.215231,0.511515,0.000000


In [ ]:
cmap = plt.cm.PuBu
corr = diff
plt.figure(figsize=(20, 16)) 
plt.title('Person Correlation - GAP Between Before COVID-19 and After COVID-19', y = 1.05, size = 15) 
sns.heatmap(corr, linewidths = 0.1, 
            vmax = 1.0, 
            square = True, 
            cmap = cmap, 
            linecolor = "white", 
            annot = True, 
            annot_kws = {"size" : 16})

In [16]:
WTI_df = pd.DataFrame({'Market Indicator' : ['BEFORE', 'AFTER', 'GAP'],
                        'Values' : [-0.27, 0.83, 1.1]})

GOLD_df = pd.DataFrame({'Market Indicator' : ['BEFORE', 'AFTER', 'GAP'],
                         'Values' : [-0.12, 0.1, 0.22]})

TB_df = pd.DataFrame({'Market Indicator' : ['BEFORE', 'AFTER', 'GAP'],
                        'Values' : [-0.49, 0.5, 0.99]})

USD_df = pd.DataFrame({'Market Indicator' : ['BEFORE', 'AFTER', 'GAP'],
                        'Values' : [-0.11, -0.62, -0.5]})

In [17]:
diff_data = pd.concat([WTI_df, GOLD_df], axis = 0)
diff_data = pd.concat([diff_data, TB_df], axis = 0)
diff_data = pd.concat([diff_data, USD_df], axis = 0)
diff_data.reset_index(drop = True, inplace = True)
kospi_diff = diff_data.loc[[2, 5, 8, 11]]
kospi_bfaf = diff_data.drop(index = [2, 5, 8, 11])

In [ ]:
plt.figure(figsize = (25, 10))
sns.barplot(data = diff_data, x = 'Market Indicator', y = 'Values', color = 'grey')
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize = (25, 10))
sns.barplot(data = kospi_diff, x = 'Market Indicator', y = 'Values')
plt.grid()
plt.show()

In [ ]:
sns.set_style('darkgrid')
fig, axes = plt.subplots(1, 4, figsize= (25, 10))
sns.barplot(x = 'Market Indicator', y = 'Values', data = WTI_df, color = 'brown', alpha = 0.7, edgecolor = 'white', ax = axes[0])
axes[0].set_ylim(-1.0, 1.2)
axes[0].set_ylabel('Coefficient')
axes[0].set_xlabel('WTI')
axes[0].set_title("KOSPI - WTI Coefficient")
axes[0].grid()

sns.barplot(x = 'Market Indicator', y = 'Values', data = GOLD_df, color = 'teal', alpha = 0.7, edgecolor = 'white', ax = axes[1])
axes[1].set_ylim(-1.0, 1.2)
axes[1].set_ylabel('Coefficient')
axes[1].set_xlabel('GOLD')
axes[1].set_title("KOSPI - GOLD Coefficient")
axes[1].grid()

sns.barplot(x = 'Market Indicator', y = 'Values', data = TB_df, color = 'blue', alpha = 0.7, edgecolor = 'white', ax = axes[2])
axes[2].set_ylim(-1.0, 1.2)
axes[2].set_ylabel('Coefficient')
axes[2].set_xlabel('BOND')
axes[2].set_title("KOSPI - BOND Coefficient")
axes[2].grid()

sns.barplot(x = 'Market Indicator', y = 'Values', data = USD_df, color = 'purple', alpha = 0.7, edgecolor = 'white', ax = axes[3])
axes[3].set_ylim(-1.0, 1.2)
axes[3].set_ylabel('Coefficient')
axes[3].set_xlabel('USD')
axes[3].set_title("KOSPI - USD Coefficient")
axes[3].grid()

plt.suptitle("Market Indicator Coefficient Comperison by KOSPI", fontsize = 25)
plt.show()

In [ ]:
wti = af_data['WTI']
reverse_idx = list(range(wti.shape[0]-1, -1, -1))
wti_reverse = pd.DataFrame(wti, index = reverse_idx)
wti_reverse.reset_index(drop=True, inplace = True)
wti_reverse

In [ ]:
wti_reverse.plot()

In [43]:
columns = ['WTI', 'KOSPI', 'GOLD', 'TB', 'USD']

def reverse(data, col):
    col = data[col]
    reverse_idx = list(range(col.shape[0]-1, -1, -1))
    reverse = pd.DataFrame(col, index = reverse_idx)
    col = reverse.reset_index(drop=True, inplace = True)
    return reverse

In [44]:
reverse_df = pd.DataFrame()

for col in columns:
    temp = reverse(af_data, col)
    reverse_df = pd.concat([reverse_df, temp], axis = 1)


reverse_df

,WTI,KOSPI,GOLD,TB,USD
0,47.18,2014.15,1642.1,1.11,1193.0
1,46.78,2059.33,1641.1,1.03,1183.0
2,45.90,2085.26,1666.4,1.05,1184.5
3,41.28,2040.22,1670.8,1.08,1191.0
4,31.13,1954.77,1674.5,1.04,1201.5
...,...,...,...,...,...
444,90.27,2707.82,1803.0,2.15,1204.5
445,92.31,2750.26,1806.6,2.19,1199.5
446,91.32,2745.06,1820.6,2.23,1200.5
447,92.81,2648.80,1925.4,2.22,1204.5


In [45]:
bf_reverse_df = pd.DataFrame()

for col in columns:
    temp = reverse(bf_data, col)
    bf_reverse_df = pd.concat([bf_reverse_df, temp], axis = 1)
bf_reverse_df

,WTI,KOSPI,GOLD,TB,USD
0,81.51,1696.14,1117.7,4.44,1153.0
1,81.77,1690.62,1118.1,4.36,1140.5
2,83.18,1705.32,1135.9,4.43,1138.5
3,82.66,1683.45,1133.1,4.32,1135.5
4,82.75,1695.26,1138.2,4.36,1132.0
...,...,...,...,...,...
2177,51.43,2079.04,1672.4,1.14,1219.0
2178,49.90,2103.61,1646.9,1.17,1215.0
2179,48.73,2076.77,1640.4,1.14,1215.0
2180,47.09,2054.89,1640.0,1.19,1212.0


In [ ]:
fig, ax = plt.subplots(2, 5, figsize = (25, 10))

reverse_df['WTI'].plot(kind='line', ax=ax[0], color='teal', title='WTI', alpha = 0.7)
ax[0].set_xlabel('Elapsed Date')
ax[0].set_ylabel('Price')
ax[0].set_title("WTI")

reverse_df['KOSPI'].plot(kind='line', ax=ax[1], color='brown', title='KOSPI', alpha = 0.7)
ax[1].set_xlabel('Elapsed Date')
ax[1].set_ylabel('Price')
ax[1].set_title("KOSPI")

reverse_df['GOLD'].plot(kind='line', ax=ax[2], color='blue', title='GOLD', alpha = 0.7)
ax[2].set_xlabel('Elapsed Date')
ax[2].set_ylabel('Price')
ax[2].set_title("GOLD")

reverse_df['TB'].plot(kind='line', ax=ax[3], color='purple', title='TB', alpha = 0.7)
ax[3].set_xlabel('Elapsed Date')
ax[3].set_ylabel('Price')
ax[3].set_title("TB")

reverse_df['USD'].plot(kind='line', ax=ax[4], color='red', title='USD', alpha = 0.7)
ax[4].set_xlabel('Elapsed Date')
ax[4].set_ylabel('Price')
ax[4].set_title("USD")

plt.suptitle("Market Indicator Price - After COVID-19", fontsize = 25)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 5, figsize = (25, 10))

bf_reverse_df['WTI'].plot(kind='line', ax=ax[0], color='teal', title='WTI', alpha = 0.7)
ax[0].set_xlabel('Elapsed Date')
ax[0].set_ylabel('Price')
ax[0].set_title("WTI")

bf_reverse_df['KOSPI'].plot(kind='line', ax=ax[1], color='brown', title='KOSPI', alpha = 0.7)
ax[1].set_xlabel('Elapsed Date')
ax[1].set_ylabel('Price')
ax[1].set_title("KOSPI")

bf_reverse_df['GOLD'].plot(kind='line', ax=ax[2], color='blue', title='GOLD', alpha = 0.7)
ax[2].set_xlabel('Elapsed Date')
ax[2].set_ylabel('Price')
ax[2].set_title("GOLD")

bf_reverse_df['TB'].plot(kind='line', ax=ax[3], color='purple', title='TB', alpha = 0.7)
ax[3].set_xlabel('Elapsed Date')
ax[3].set_ylabel('Price')
ax[3].set_title("TB")

bf_reverse_df['USD'].plot(kind='line', ax=ax[4], color='red', title='USD', alpha = 0.7)
ax[4].set_xlabel('Elapsed Date')
ax[4].set_ylabel('Price')
ax[4].set_title("USD")

plt.suptitle("Market Indicator Price - Before COVID-19", fontsize = 25)
plt.show()